# 시작하기 전에

---

현재까지 배운 모델은  
* k-neighbors regression/classifier  
* linear regression  
* ridge regression  
* lasso regression  
* classifier with SGD(stochastic Gradient Descent)  
* DecisionTree classifier

와 같다.  

가장 좋은 알고리즘이 있다고 해서 다른 알고리즘을 학습하지 않을 이유는 없다 해결해야 하는 문제마다 최적의 알고리즘이 있고 같은 Classifier이나 Regression에도 최적화된 방법이 있기 때문.

# 정형 데이터와 비정형 데이터

---

pandas를 통해서 pd.read_csv 를 통해 DataFrame을 불러오게 된다면

|feature1|feature2|feature3|
|---|---|---|
|value1|value2|value3|  

이렇게 나오게 되는데 이런 형태의 데이터를 ***정형 데이터 (structured data)***라고 부른다.

쉽게 말해 구조석으로 설계된 데이터인데 이런 데이테는 CSV나 DataBase, Excel 등에 저장하기 쉽다.

이와 반대되는 데이터를 비정형 데이터(unstrucred data)라고 하며 텍스트 데이터나 디지털 카메라로 찍은 사진 음악과 같은 데이터들이 속하며, DB나 Excel에 표현하기 어려운 것들이다.

지금까지 배운 머신러닝 알고리즘은 정형 데이터에 잘 맞는다. 그 중에 정형 데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘이 앙상블 학습(ensemble learning)이다.

이 알고리즘은 대부분 걸졍트리를 기반으로 만들어져 있이.

> 비정형 데이터의 경우에는 7장에서 배울 신경망 알고리즘을 이용해서 학습한다.  
비정형 데이터는 규칙성을 찾기 어려워 전통적인 머신러닝 방법으로는 모델을 만들기 까다롭기 때문이다.

#DecisionTree-Based ensemble learning

## 랜덤 포레스트(Random Forest)

---

랜덤 포레스트는 앙상블 학습의 대표 주자 중 하나로 안정적인 성능 덕분에 널리 사용되고 있다.
> 앙상블 학습을 적용할 때 가장 먼저 랜덤 포레스트를 시도해보길 권장.

랜덤 포레스트 이름 자체로 유추할 수 있듯이 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만든다. 그리고 각 결정 트리의 예측을 사용해 최죵 예측을 만든다.

---

먼저 랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데, 이 데이터를 만드는 방법이 독특하다. 우리가 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만든다. 이때 한 샘플이 중복되어 추출될 수도 있다.

이와 같은 방법을 부트스트랩(bootstrap sample)이라 부르는데 일반적인 resampling이다.

> 부트스트랩은 복원 추출의 개념을 적용하는데 추출한 1개의 샘플을 다시 모집단에 넣고 뽑는과정을 반복한다.  
N개의 데이터가 있다고 가정한다면, 이 과정을 N번 반복하여 Bootstrap set을 하나더 만든다.

> 각 노드를 분할할 때 전체 특정 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾는다.  
* 분류 모델인 RandomForestClassifier 는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택한다.  
만약 훈련세트에 4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택하여 사용.
* 회귀 모델인 RandomForestRegressor 는 전체 특성을 사용한다.  
* sklearn의 랜덤 포레스트는 기본적으로 100개의 DT를 이런 방식으로 훈련한다.  
Classifier의 경우에는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼고.  
Regressor 의 경우에는 단순히 각 트리의 예측을 평균한다.

랜덤 포레스트의 경우 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련세트에 과대적합 되는것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.

종종 기본 매개변수 설정만으로(하이퍼 파라미터 튜닝없이) 좋은 결과를 낸다.

## 실습

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

wine = pd.read_csv("http://bit.ly/wine_csv_data")

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()

target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target,test_size = 0.2, random_state = 42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# cross_validate에 넣어줄 모델 객체 설정하기

rf = RandomForestClassifier(n_jobs = -1, random_state = 42)

scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)


print(np.mean( scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


랜덤 포레스트의 경우 DecisionTree의 ensemble 이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공한다.

criterion, max_depth, max_features, min_sample_split...

또한 결정 트리의 큰 장점중 하나인 특정 중요도를 계산한다.

In [3]:
rf.fit(train_input, train_target)

print(rf.feature_importances_)


[0.23167441 0.50039841 0.26792718]


In [4]:
# Decision_Tree를 사용했을 때와의 특성 중요도 차이 확인

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth = 3, random_state = 42)

dt.fit(train_input, train_target)
print(dt.feature_importances_)

[0.12345626 0.86862934 0.0079144 ]


DT를 이용해서 특성 중요도를 확인했을 때는 당도를 뜻하는 sugar의 특정 중요도가 86%였던 때와 다르게 산도와 알코올 도수 역시 큰 영향을 주는것을 확인할 수 있었다.

이런 현상이 나타나는 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정트리를 훈련하기 때문이다. 하나의 특성에 과도하게 집중하지 않고 랜덤으로 특성을 선택하여 노드를 이어나가기 때문에 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다. 이는 과대적합을 줄이고 일반화 성능을 높이는데 도움이 된다.

RandomForestClassifier에는 자체적으로 모델을 평가하는 점수를 얻을 수 있다.

랜덤 포레스트는 BootStrap 샘플을 만들어서 결정 트리를 훈련하는데, 이때 중복 추출로 인해서 추출되지 않은 샘플을 OOB(out of box) 샘플이라고 한다. 이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정트리를 평가할 수 있다. 마지 검증 세트의 역할을 하는것.

> 확률상 훈련세트의 $\frac{2}{3}$가 추출된다고 한다. 따라서 확률적으로 OOB는 훈련세트의 $\frac{1}{3}$가 된다.

이 점수를 얻기 위해서는 RandomForestClassifier 클래스의 oob_score 매개변수를 True로 지정해야 한다.(default = False) 이렇게 하면 랜덤 포레스트는 각 결정 트리의 OOB 점수를 평균하여 출력한다.

In [5]:
rf = RandomForestClassifier()

rf.oob_score = True
rf.n_jobs = -1
rf.random_state = 42

rf.fit(train_input, train_target)

print(rf.oob_score_)

0.8934000384837406


oob_score_에서의 결과값은 cross_validate에서 얻은 교차검증의 교차검증 점수의 평균값인  
```np.mean(scores['test_score'])```
와 비슷한것을 확인할 수 있다.

oob점수는 교차 검증을 대신할 수 있어서 결과적으로 훈련세트에 더 많은 샘플을 사용할 수 있다.

## 엑스트라 트리(Extra Tree)

---

***엑스트라 트리*** 는 랜덤 포레스트와 매우 비슷하게 동작한다. 기본적으로 100개의 결정트리를 훈련한다. 랜덤 포레스트와 동일하게 결정 트리가 제공하는 대부분의 매개변수를 지원한다. 또한 전체 특정 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는데 사용한다.

랜덤 포레스트와 엑스트라 트리의 차이점은 BootStrap을 사용하지 않는다는 점이다.

즉각 결정 트리를 만들 때 전체 훈련 세트를 사용한다. 대신 노드를 분할 할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할한다.

Extra Tree의 경우에는 DecisionTree의 splitter = 'random'을 사용한다.

>DecisionTree 매개변수에서 spliiter에 대해서 알아봤었는데,  
부모 노드와 자식 노드의 불순도 차이를 나타내는 정보이득을 나타낸다.

하나의 결정 트리에서 특성을 무작위로 분할한다면 선으이 낮아지겠지만 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있다. sklearn에서 제공하는 ExtraTree는 ExtraTreesClassifier이다.



In [6]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier()
et.n_jobs = -1
et.random_state = 42

scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 한다. 하지만 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 엑스트라 트리의 장점이다.

> DecisionTree는 최적의 분할을 찾는데 시간을 많이 소모한다. 특히 고려해야할 특성의 개수가 많을 때 더 그렇다. 만약 무작위로 나눈다면 훨씬 빠르게 트리를 구성할 수 있다.

엑스트라 트리 역시 특성 중요도를 제공한다.  
처음 wine_set를 만들 때 ['alcohol', 'sugar', 'pH']로 가져왔기 때문에 중요도의 순서 역시 다음과 같다.

In [7]:
et.fit(train_input, train_target)

print(et.feature_importances_)
print(dt.feature_importances_)

[0.20183568 0.52242907 0.27573525]
[0.12345626 0.86862934 0.0079144 ]


## 그레디언트 부스팅(Gradient Boosting)

---

Gradient Boosting은 기본적으로 Gradient인 경사 하강법을 사용한다.  
* 분류에서는 Logistic Loss function,  
회귀에서는 Mean Squared Error function을 사용한다.

깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보안하는 방식으로 앙상블 하는 방법이다. sklearn의 ***GradientBoostingClassifier*** 은 기본적으로 깊이가 3인 결정 트리를 100개 사용한다.
> max_depth = 3인 DecisionTree 100개를 통해서 오차를 줄여나가는 방식을 사용  

깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있다.

이전에 경사 하강법은 손실 함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정이고, 이때 가장 낮은 곳을 찾아 내려오는 방법은 모델의 가중치와 절편을 조금씩 바꾸는 것이라고 했다.

그레디언트 부스팅은 결정 트리를 계속 추가하면서 Loss가 가장 낮은 곳을 찾아 이동한다. 그래서 깊이가 얕은 트리를 사용하고 또 학습률 매개변수로 속도를 조절한다.



In [8]:
# GradientBoostingClassifier 사용 DataSet cross-validation score확인하기

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb.random_state = 42

scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


현재까지 ensemble learning을 진행하는 동안 처음으로 과대적합이 발생하지 않고 가장 좋은 정확도를 보여주었다. 만일 DT의 갯수를 늘리거나 학습률을 다르게 가져간다면 더 좋은 결과가 나타날 수 있을 것이다.

* GradientBoostingClassifier의 매개변수  
  * learning_rate  
  학습률
  default = 0.1
  ---
  * n_estimators  
  결정 트리의 개수
  default = 100  
  ---
  * subsample  
  트리 훈련에 사용할 훈련 세트의 비율
  default = 1.0  
  > subsample이 1보다 작으면 훈련 세트의 일부를 사용한다.  
  이는 마치 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택하는
  확률적 경사 하강법(SGB), 미니배치 경사 하강법과 비슷하다.

  

In [9]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42)

scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [10]:
# 특성 중요도 확인
gb.fit(train_input, train_target)

print(gb.feature_importances_)

print(dt.feature_importances_)

[0.15872278 0.68010884 0.16116839]
[0.12345626 0.86862934 0.0079144 ]


그레디언트 부스팅의의 단점은 순서대로 트리를 추가하여 오차를 줄여나가는 방식이여서 다중 연산에 필요한 CPU의 코어 수를 조절하는 n_jobs가 없다.  
때문에 훈련속도가 다소 느리다는 단점이 존재한다.

> Gradient_Boosting의 Regression은 GradientBoostingRegressor 이다.

# 히스토그램 기반 그레디언트 부스팅(Histogram-based Gradient Boostring)

---

***히스토그램 기반 그레디언트 부스팅*** (이하 hgb)은 정형 데이터를 다루는 머신러닝 알고리즘 중 가장 인기가 높은 알고리즘이다.

이 모델은 먼저 입력 특성을 256개의 구간으로 나누기 때문에 최적의 분할을 매우 빠르게 찾을 수 있다.  
히스토 그램 기반 그레디언트 부스팅은 256개의 구간 중에서 하나를 떼어놓고 누락된 값을 위해서 사용한다. 따라서 <U>입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없다.</U>

## sklearn HistGradientBoostingClaasifier

---

sklearn의 hgb 클래스는 HistGradientBoostingClassifier이다.  
다른 ensemble learning과 마찬가지로 기본 매개변수 만으로 안정적인 성능을 얻을 수 있다.



> gb에서의 결정 트리 개수를 나타냈던 n_estimator 대신 max_iter를 사용한다.

hgb의 경우 sklearn에서 테스트 과정에 있기 때문에 이 클래스를 사용하려면 sklearn.experimental 패키지 안에 enable_hist_gradient_boosting 모듈을 임포트 해줘야한다.

In [12]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state = 42)

scores = cross_validate(hgb, train_input, train_target, return_train_score = True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


과대적합을 작 억제하면서 gb보다 조금 더 높은 성능을 제공한다.

특성 중요도를 확인할 때 .feature_importances_가 아닌 permutation_importance() 함수를 사용한다.

* permutation_importance() 의 매개변수
  * n_repeats  
  특성을 랜덤하게 섞을 횟수 지정  
  
  ---

> 이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산한다. 훈련 세트뿐만 아니라 테스트 세트에도 적용할 수 있고 sklearn에서 제공하는 추정기(estimator) 모델에 모두 사용할 수 있다.  



In [16]:
# n_repeats 매개변수를 통해서 랜덤하게 섞을 횟수를 지정한다.
from sklearn.inspection import permutation_importance


hgb.fit(train_input, train_target)


result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)

print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [17]:
# test_set에서 특성 중요도 확인하기

result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state = 42, n_jobs = -1)

print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [20]:
# 최종 점수 확인

print(hgb.score(test_input, test_target))

0.8723076923076923


확실하게 ensemble_learning에서는 무려 하이퍼파라미터 튜닝을 거친 DecisionTree보다 높은 정확도를 보여주는것을 확인할 수 있었다.

## XGBoost

---

XGBoost는 히스토그램 기반 그레디언트 부스팅의 가장 대표적인 라이브러리이다.  
놀랍게도 코랩에서 사용할수 있을 뿐만 아니라 cross_validate()함수 역시 사용 가능하다.

> XGBoost는 다양한 부스팅 알고리즘을 지원한다. tree_method 매개변수를 'hist'로 지정하면 히스토그램 기반 그레디언트 부스팅을 사용할 수 있다.

In [21]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method = 'hist', random_state = 42)

scores = cross_validate(xgb, train_input, train_target, return_train_score = True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))


0.9558403027491312 0.8782000074035686


## LightGBM

---

MS soft에서 나온 LightGBM은 빠르고 최신 기술을 많이 적용하고 있어 인기가 점점 높아지고 있다. 추후 기회가 된다면 더 학습할 예정

In [22]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state = 42)

scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


강의에서 본것보다 더 높은 정확도가 나왔는데 계속 발전하고 있는건지는 정확하게 모르겠다.

#고찰

---

이번 절에서는 ensemble_learning에 대해서 학습하였다.

ensemble_learning 에서 결정트리 기반의 과 히스토그램 기반의 학습의 모델들에 대해서 알아보았다.

* Decision_tree - based
  * 랜덤 포레스트  
  랜덤 포레스트의 경우 가장 대표적인 ensemble_learning  algorithm중 하나이다. 성능이 좋고 안정적이기 때문에 task를 마주했을 때 첫 번째로 시도해볼 수 있는 학습 중 하나이다.  
  > 랜덤 포레스트는 결정트리를 훈련하기 위해서 부트스트랩 샘플을 만들고 전체 특성중 일부를 랜덤하게 선택하여 결정트리를 만든다.  
  랜덤 포레스트의 경우 확률적으로 전체 훈련세트에서 $\frac{2}{3}$ 정도를 부트스트랩에 사용하는데 나머지 $\frac{1}{3}$인 OOB(Out Of Box)데이터를 이용해서 검증세트를 대신하는 결과를 얻을 수도 있었다.
  ---

  * 엑스트라 트리  
  엑스트라 트리는 랜덤포레스트와 매우 비슷하지만 부트스트랩 샘플을 사용하지 안혹 노드를 분할할 때 최선이 아니라 랜덤하게 분할한다. 이런 특징 때문에 훈련 속도가 빠르지만 보통 더 많은 트리가 필요하다.  
  ---

  * 그레디언트 부스팅  
  그레디언트 부스팅은 깊이가 얕은 트리를 연속적으로 추가하여 손실 함수를 최소화하는 앙상블 방법이다.  
  성능은 뛰어나지만 결정트리를 하나씩 이어나가면서 학습하기때문에 속도가 다소 느리다는 단점이 있다.

  > 그레디언트 부스팅에서 학습률 매개변수(learning_rate)를 조정하여 모델의 복잡도를 제어할 수 있다.

---

* Histogram_based Gradient Boosting  
  * 히스토그램 기반 그레디언트 부스팅은 훈련 데이터를 256개의 구간으로 변환하여 사용하기 때문에 노드 분할 속도가 매우 빠르다.  
  코랩에는 sklearn 뿐만 아니라 다른 라이브러리 역시 지원하기 때문에 추후에 XGBoost와 LightGBM라이브러리에 대해서 학습할 예정이다.